In [ ]:
!pip install -qU langchain\
    langchain-community\
    langchain-together\
    langchain-core\
    faiss-cpu\
    faiss-gpu\
    sentence-transformers\
    pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 k

In [ ]:
import os
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_together import ChatTogether
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from IPython.core.display import Markdown
from langchain_core.prompts import PromptTemplate
from langchain.document_loaders import PyMuPDFLoader

In [ ]:
os.environ["TOGETHER_API_KEY"] = "c831603b49af1a3eaf681514d22fd8af442f6790b490d463d351d5c318ad2540"

In [ ]:
# Loading the ruleset file

pdf_file_paths = [
    "/content/sample_reports/RM1_4022_HW2_22.pdf" #
]

documents = []
for pdf_file in pdf_file_paths:
    if pdf_file in pdf_file_paths:
        loader = PyMuPDFLoader(file_path=pdf_file)
        l = loader.load()
        documents.extend(l)

### retriever

retriever is not used in this proof of concept.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=32)
chunks = text_splitter.split_documents(documents)

embedding_function = HuggingFaceEmbeddings(show_progress=True, multi_process=True)
vector_store = FAISS.from_documents(documents=chunks, embedding=embedding_function)

<ipython-input-128-32b82e04eb42>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = HuggingFaceEmbeddings(show_progress=True, multi_process=True)
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


KeyboardInterrupt: 

In [ ]:
!pip install rank_bm25
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [ ]:
bm25_retriever = BM25Retriever.from_documents(chunks)
faiss_retriever = vector_store.as_retriever()
retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.25, 0.75]
)

ValueError: not enough values to unpack (expected 2, got 0)

we can now define a query and use the retriever to retrieve.


In [ ]:
# example usage of retrieving documents which demonstrates that it does work with Farsi data

query = "abstract"

retrieved_results = retriever.get_relevant_documents(query)
retrieved_results

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'source': '/content/ruleset.pdf', 'file_path': '/content/ruleset.pdf', 'page': 1, 'total_pages': 2, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS Version 12.2.1 (Build 21D62) Quartz PDFContext', 'creationDate': "D:20240821102145Z00'00'", 'modDate': "D:20240821102145Z00'00'", 'trapped': ''}, page_content='application ھﺎﯾﯽ ﮐﮫ ﻗﺎﺑﻞ اﺳﺘﻔﺎده ھﺴﺘﻨﺪ ذﮐﺮ ﺷﻮﻧﺪ.ﺑﮫ طﻮر ﻣﺜﺎل \n \nداده ھﺎی ﺧﺎم و داده ھﺎی ﭘﺮدازش ﺷﺪه ﺑﺎﯾﺪ در reuslt ﺑﺎﺷﻦ و ﻧﮫ .conclusion \n \n.1 اﯾﻦ ﻗﺴﻤﺖ از ﭼﮑﯿﺪه ﻧﺸﺎن دھﻨﺪه ی ارزش و ﺟﺎﯾﮕﺎه ﺗﺤﻘﯿﻖ ﻣﯽ ﺑﺎﺷﺪ. ﺑﻨﺎﺑﺮاﯾﻦ ﺑﮫ ﻋﻠﺖ ﻋﺪم ﻗﻄﻌﯿﺖ ﻧﻮﯾﺴﻨﺪه از اﻓﻌﺎل \nﻣﻀﺎرع اﻟﺘﺰاﻣﯽ و ﯾﺎ ﺣﺘﯽ آﯾﻨﺪه ﺑﺎﯾﺪ اﺳﺘﻔﺎده ﮐﻨﺪ. \n \n.2 ﺑﯿﺸﺘﺮ آﻧﮑﮫ ﻧﺘﺎﯾﺞ ﻧﻤﺎﯾﻨﺪه ی ﻓﺮﺳﺘﻨﺪه ﻣﯽ ﺑﺎﺷﺪ و ﺟﻤﻊ ﺑﻨﺪی ﻣﺎﻧﻨﺪ ﮔﯿﺮﻧﺪه.flasher ھﺎﯾﯽ ﮐﮫ در اﯾﻨﺠﺎ ﭘﺮ اﺳﺘﻔﺎده ﺧﻮاھﺪ ﺑﻮد \nﻓﻌﻞ conclude ﯾﺎ contributes ﻣﯽ ﺑﺎﺷﻨﺪ. \n \nﻗﺎﻧﻮن ﺷﺸﻢ: ﮐﻠﯿﺖ ﭼﮑﯿﺪه ﺑﺎﯾﺪ ﺑﮫ ﺻﻮرت زﯾﺮ ﺑﺎﺷﺪ:'),
 Document(metadata={'source': '/content/ruleset.pdf', 'file_path': '/content/ruleset.pdf', 'page

### model initialization

In [ ]:
llm = ChatTogether(
    model="meta-llama/Llama-3-70b-chat-hf",
    temperature= 0
    )

first step: generating a grading system to grade the given abstracts with.

In [ ]:
generate_system_template = (
    "you are a professional reviewer."
    "you are asked to evaluate a written abstract of an article based on a ruleset."
    "based on the given ruleset, propose a grading system."
    "the total grade should sum up to 20."
    "ruleset: {ruleset}"
)
generate_system_prompt = ChatPromptTemplate.from_template(generate_system_template)

In [ ]:
generate_system_chain = generate_system_prompt | llm | StrOutputParser()

In [ ]:
grade_system = generate_system_chain.invoke(
    {
        "ruleset": documents,
    }
)

BadRequestError: Error code: 400 - {'error': {'message': 'Input validation error: `inputs` tokens + `max_new_tokens` must be <= 8193. Given: 8602 `inputs` tokens and 1 `max_new_tokens`', 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': None}}

In [ ]:
print(grade_system)

Based on the provided ruleset, I propose a grading system to evaluate the written abstract. The total grade will sum up to 20.

**Grading Criteria:**

1. **Location and Structure** (2 points):
	* Is the abstract located at the beginning of the article?
	* Does the abstract follow the recommended structure (background, purpose, methodology, results, and conclusion)?
2. **Keywords and Phrasing** (2 points):
	* Are the provided keywords incorporated naturally into the abstract?
	* Is the language concise, clear, and free of negative verbs and jargon?
3. **Instructive and Brief** (2 points):
	* Does the abstract accurately reflect the content of the article?
	* Is the abstract brief and within the recommended word limit (100-125 words)?
4. **Self-Sufficient** (2 points):
	* Is the abstract understandable on its own, without referring the reader to the full article?
	* Are undefined terms or acronyms avoided?
5. **Clear and Well-Written** (2 points):
	* Is the language engaging and informat

second step: score the abstracts with the grading system.

In [ ]:
generate_grade_template = (
    "you are a professional reviewer."
    "you are asked to evaluate a written abstract of an article based on the following ruleset."
    "based on the given grading system grade the given abstract."
    "be super strict. They will either get all the points of each section, or they get none of it"
    "grading_system: {grading_system}"
    "abstract: {abstract}"
)
generate_grade_prompt = ChatPromptTemplate.from_template(generate_grade_template)

In [ ]:
generate_grade_chain = generate_grade_prompt | llm | StrOutputParser()

the abstract to be evaluated is now defined

In [ ]:
abstract = """
Recent advancements in large language models (LLMs) on language modeling and emergent capabilities make them a promising reference-free evaluator of nat- ural language generation quality, and a competent alternative to human evaluation. However, hindered by the closed-source or high computational demand to host and tune, there is a lack of practice to further calibrate an off-the-shelf LLM-based evaluator towards better human alignment. In this work, we propose AUTOCALI- BRATE, a multi-stage, gradient-free approach to automatically calibrate and align an LLM-based evaluator toward human preference. Instead of explicitly model- ing human preferences, we first implicitly encompass them within a set of human labels. Then, an initial set of scoring criteria is drafted by the language model itself, leveraging in-context learning on different few-shot examples. To further calibrate this set of criteria, we select the best performers and re-draft them with self-refinement. Our experiments on multiple text quality evaluation datasets il- lustrate a significant improvement in correlation with expert evaluation through calibration. Our comprehensive qualitative analysis conveys insightful intuitions and observations on the essence of effective scoring criteria.
"""

In [ ]:
grade = generate_grade_chain.invoke(
    {
        "grading_system": grade_system,
        "abstract": abstract
    }
)

In [ ]:
print(grade)

I will evaluate the abstract based on the provided grading system. Here's my assessment:

**Location and Structure** (2 points): 2 points
The abstract is located at the beginning of the article, and it follows the recommended structure (background, purpose, methodology, results, and conclusion).

**Keywords and Phrasing** (2 points): 2 points
The provided keywords (e.g., "large language models", "natural language generation quality", "human alignment") are incorporated naturally into the abstract. The language is concise, clear, and free of negative verbs and jargon.

**Instructive and Brief** (2 points): 2 points
The abstract accurately reflects the content of the article, and it is brief and within the recommended word limit (100-125 words).

**Self-Sufficient** (2 points): 2 points
The abstract is understandable on its own, without referring the reader to the full article. Undefined terms or acronyms are avoided.

**Clear and Well-Written** (2 points): 2 points
The language is engag

another example where the abstract does not have the results section.

In [ ]:
abstract = """
Recent advancements in large language models (LLMs) on language modeling and emergent capabilities make them a promising reference-free evaluator of nat- ural language generation quality, and a competent alternative to human evaluation. However, hindered by the closed-source or high computational demand to host and tune, there is a lack of practice to further calibrate an off-the-shelf LLM-based evaluator towards better human alignment. In this work, we propose AUTOCALI- BRATE, a multi-stage, gradient-free approach to automatically calibrate and align an LLM-based evaluator toward human preference. Instead of explicitly model- ing human preferences, we first implicitly encompass them within a set of human labels. Then, an initial set of scoring criteria is drafted by the language model itself, leveraging in-context learning on different few-shot examples. To further calibrate this set of criteria, we select the best performers and re-draft them with self-refinement.
"""

In [ ]:
grade = generate_grade_chain.invoke(
    {
        "grading_system": grade_system,
        "abstract": abstract
    }
)
print(grade)

I will evaluate the abstract based on the provided grading system. Here's my assessment:

**Location and Structure** (2 points): 2 points
The abstract is located at the beginning of the article, and it follows the recommended structure (background, purpose, methodology, and results).

**Keywords and Phrasing** (2 points): 2 points
The abstract incorporates keywords like "large language models," "natural language generation quality," and "human alignment" naturally. The language is concise and clear, with no negative verbs or jargon.

**Instructive and Brief** (2 points): 2 points
The abstract accurately reflects the content of the article, and it is brief, within the recommended word limit (100-125 words).

**Self-Sufficient** (2 points): 2 points
The abstract is understandable on its own, without referring the reader to the full article. Undefined terms or acronyms are avoided.

**Clear and Well-Written** (2 points): 2 points
The language is engaging and informative, with strong verbs